#### [다변량 데이터기반 회귀 모델]
- 주제 : 학습 시간에 따른 수능 점수 예측 서비스
- 데이터 : study_score_multi.csv
- 구성 : 피쳐 + 타겟
- 학습 : 지도학습 + 회귀
- 구현 : 인공신경망


[1] 모듈로딩 및 데이터 준비<hr>

In [1]:
# %pip install visdom

In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchinfo import summary

In [3]:
data_file = '../Data/study_score_multi.csv'

## 데이터 로딩
dataDF = pd.read_csv(data_file)
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   study_hours    5000 non-null   float64
 1   sleep_hours    5000 non-null   float64
 2   participation  5000 non-null   float64
 3   score          5000 non-null   float64
dtypes: float64(4)
memory usage: 156.4 KB


[2] ANN 모델 설계 <hr>

In [4]:
## ==========================================================
##          입력수      퍼셉트론수/출력수       AF
## ==========================================================
## 입력층     3              3               ★Pytorch에는 입력층(클래스) X, 입력 텐서를 입력층으로 간주
## 은닉층     3              10              ReLU
## 은닉층     10             15              ReLU
## 출력층     15             1               X(회귀라서)
## ==========================================================
## 클래스이름 : ScoreModel
## 부모클래스 : nn.Module
## 오버라이딩 : __init__(self)  : 층 구성 요소 인스턴스 생성
##            forward(self, x) : 순전파 진행 메서드, ★ x:입력층
## ==========================================================
class ScoreModel(nn.Module) :
    ## 층 구성 인스턴스 생성 메서드
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.hd1_layer= nn.Linear(3, 10)
        self.hd2_layer= nn.Linear(10, 15)
        self.out_layer= nn.Linear(15, 1)

    ## 순전파 진행 메서드 
    def forward(self, x) :
        ## input -> hd1 : 1 -> 10
        out = self.hd1_layer(x)
        out = F.relu(out)
        
        ## hd1 -> hd2 : 10 -> 15
        out = self.hd2_layer(out)
        out = F.relu(out)
        
        ## hd2 -> out : 15 -> 1
        ## 회귀모델로 AF없음
        out = self.out_layer(out)




In [ ]:
## 모델 객체 생성
model = ScoreModel()
summary(model, input_size=(1000,3))

Layer (type:depth-idx)                   Output Shape              Param #
ScoreModel                               --                        --
├─Linear: 1-1                            [100000000, 10]           40
├─Linear: 1-2                            [100000000, 15]           165
├─Linear: 1-3                            [100000000, 1]            16
Total params: 221
Trainable params: 221
Non-trainable params: 0
Total mult-adds (G): 22.10
Input size (MB): 1200.00
Forward/backward pass size (MB): 20800.00
Params size (MB): 0.00
Estimated Total Size (MB): 22000.00